In [1]:
import pystac
from pathlib import Path
from shapely.geometry import Polygon, mapping

import pandas as pd
from rasterio.warp import transform_bounds, transform_geom

import geopandas as gpd
from datetime import datetime, timezone


In [2]:
catalog = pystac.Catalog(
    id='forecast-test',
    title="Landsat FOVs",
    description='This is a test of Landsat image forecasts',
    stac_extensions=['https://stac-extensions.github.io/projection/v1.0.0/schema.json']
)

In [3]:
grid = gpd.read_file('./tmp/n_visits.geojson')
bbox = grid.total_bounds
print(grid.crs.to_string())
print(bbox)

EPSG:4326
[-81.41094255 -55.61183    -34.41094255  13.38817   ]


In [4]:
def get_bbox_and_footprint(bounds):

    # create the bounding box (not really needed for gpd - is lat/ lon right?)
    bbox = bounds

    # create the footprint
    footprint = Polygon([
        [bbox[0], bbox[1]],
        [bbox[0], bbox[3]],
        [bbox[2], bbox[3]],
        [bbox[2], bbox[1]]
    ])

    return bbox, mapping(footprint)

In [5]:
get_bbox_and_footprint(grid.total_bounds)

(array([-81.41094255, -55.61183   , -34.41094255,  13.38817   ]),
 {'type': 'Polygon',
  'coordinates': (((-81.41094255239946, -55.61183),
    (-81.41094255239946, 13.38817),
    (-34.41094255239946, 13.38817),
    (-34.41094255239946, -55.61183),
    (-81.41094255239946, -55.61183)),)})

In [6]:
folder = Path('./tmp/')
file_list = list(folder.rglob('*fovs.geojson'))

fovs = gpd.read_file(file_list[0])

fovs

,satellite,id,time,lonspan,color,geometry
0,LANDSAT 8,139084,2021-12-10T11:36:00,4.708169,#ff0000ff,"POLYGON ((-30.59647 -53.14575, -33.31647 -52.6..."
1,LANDSAT 8,139084,2021-12-10T12:56:00,2.479858,#ff0000ff,"POLYGON ((-37.94314 14.90338, -39.62978 15.260..."
2,LANDSAT 8,139084,2021-12-10T12:57:00,2.444997,#ff0000ff,"POLYGON ((-38.75557 11.28052, -40.41769 11.635..."
3,LANDSAT 8,139084,2021-12-10T12:58:00,2.422150,#ff0000ff,"POLYGON ((-39.55042 7.65597, -41.19535 8.00918..."
4,LANDSAT 8,139084,2021-12-10T12:59:00,2.410751,#ff0000ff,"POLYGON ((-40.33246 4.03050, -41.96721 4.38252..."
...,...,...,...,...,...,...
157,LANDSAT 9,149260,2021-12-11T14:43:00,3.515689,#0000ffff,"POLYGON ((-74.13549 -41.49507, -76.32620 -41.0..."
158,LANDSAT 9,149260,2021-12-11T14:44:00,3.793507,#0000ffff,"POLYGON ((-75.28933 -45.06404, -77.61055 -44.6..."
159,LANDSAT 9,149260,2021-12-11T14:45:00,4.137264,#0000ffff,"POLYGON ((-76.55755 -48.62215, -79.03380 -48.1..."
160,LANDSAT 9,149260,2021-12-11T14:46:00,4.568451,#0000ffff,"POLYGON ((-77.97371 -52.16732, -80.63510 -51.7..."


In [7]:
for f in file_list:
    fovs = gpd.read_file(f)
    for i in range(len(fovs[:2])):
        
        fov = fovs.iloc[i]

        bounds = fov.geometry.bounds
        bbox, footprint = get_bbox_and_footprint(bounds)

        # Project to WGS84 to obtain in geometric coordinates
        geo_bounds = transform_bounds(fovs.crs.to_string(), 'EPSG:4326', *bbox)
        geo_footprint = transform_geom(fovs.crs.to_string(), 'EPSG:4326', footprint)

        dt = datetime.fromisoformat(fov.time)
        dtz = dt.astimezone(timezone.utc)

        item = pystac.Item(
            # id=idx,
            id = str(i),
            geometry=geo_footprint,
            bbox=geo_bounds,
            datetime = dtz,
            stac_extensions=['https://stac-extensions.github.io/projection/v1.0.0/schema.json'],
            properties=dict(
                # tile=tile
            )
        )

        catalog.add_item(item)

print(len(list(catalog.get_items())))
catalog.describe()

2
* <Catalog id=forecast-test>
  * <Item id=0>
  * <Item id=1>


In [8]:
fov.time

'2021-12-10T12:56:00'

In [9]:
# ! git clone https://github.com/stac-utils/pystac.git

In [10]:
catalog.describe()

* <Catalog id=forecast-test>
  * <Item id=0>
  * <Item id=1>


In [11]:
catalog.normalize_hrefs('./stac_catalog')
catalog.save()

In [12]:
from pystac import Catalog, get_stac_version
root_catalog = Catalog.from_file('./stac_catalog/catalog.json')
print(f"ID: {root_catalog.id}")
print(f"Title: {root_catalog.title or 'N/A'}")
print(f"Description: {root_catalog.description or 'N/A'}")

ID: forecast-test
Title: Landsat FOVs
Description: This is a test of Landsat image forecasts


In [13]:
item = catalog.get_item("1", recursive=True)


In [14]:
item.geometry

{'type': 'Polygon',
 'coordinates': [[(-40.42299424761815, 11.255596061458949),
   (-40.42299424761815, 15.260761206159888),
   (-37.94313663718168, 15.260761206159888),
   (-37.94313663718168, 11.255596061458949),
   (-40.42299424761815, 11.255596061458949)]]}

In [15]:
print(item.collection_id)

None
